In [1]:
#identify which querys are question querys
#delete them from the list but keep information
#rank the querys with bm25
#rank the question queries seperately:
    #gpt generates sub querys
    #bm25.search() for each sub query
    #aggregate score
    #update original query score

#always keep track of information qid, docid, docno etc. 

In [2]:
#imports
import openai
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [3]:
client = openai.OpenAI() #connect to OpenAI API

In [4]:
def ask_gpt(prompt, model="gpt-4", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

In [5]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [13]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [7]:
querys = pt_dataset.get_topics('query')

## Identify question queries

In [8]:
#gpt goes through each query, question querys qid will be stored in answers (list)
answers = list()
for i in range(len(querys)):
    determine_abbrevation = f""" 
    You are an scientific expert especially in the domain of Information Retrieval. Your task is to detect whether
    a given query, which is given as a text below delimited by triple quotes, is a question. You can detect whether a query
    is a question if it contains question words like 'What' or 'Why'.
    For example given a query 'What is crx' you should answer yes, since the query contains 'What' which is a 
    question word. However if the given query is 'Information Retrieval' you should answer no, since there is no
    question word in the query.

    query: '''{querys['query'][i]}'''
    """
    answer = ask_gpt(prompt=determine_abbrevation) #check answer more carefully perhaps model will return not only {yes,no}
    #print(answer)
    qid = str(querys['qid'][i])
    if "yes" in answer.lower().strip():
        answers.append(qid)
print(answers)

['56', '59', '60']


### Cut question queries out of dataframe

In [9]:
for i in range(len(querys)):
    if querys['qid'][i] in answers:
        print(querys['qid'][i])
        index = querys[querys['qid'] == querys['qid'][i]].index
        querys = querys.drop(index)

56
59
60


In [10]:
#illustrate querys have been deleted
querys[53:57]

,qid,query
53,55,bm25
55,57,principle of a information retrieval indexing
56,58,architecture of web search engine
59,61,efficient retrieval algorithms


rank queries with bm25

In [16]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [17]:
#create first ranking, will later 'merge' with run2 which contains ranked question querys 
run1 = bm25(querys)
run1.head(3)

,qid,docid,docno,rank,score,query
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness


## rank question queries seperately

In [18]:
#reset querys since now we need to find the previous deleted querys
querys = pt_dataset.get_topics('query')

In [30]:
#todo: save all infos in dataframe of query? or do I need to do that when ranking is done?
questions = list()
for qid in answers:
    for i in range(len(querys)):
        if querys['qid'][i] == qid:
            query= querys['query'][i]
            print("Old query: ",query)
            #ask gpt to expand query
            expand = f""" 
            You are an expert in Information Retrieval.  I am building an retrieval System specificially for the scientific domain, just like Google Scholar I want to find scientific papers for a query,. I want to improve the precision of my retrieval system by expanding specific question querys. Please understand that the information need behind a question query is much different than for a keyword query. For example the intent behind a question query like "What is Deep Learning" much more likely is to find papers that focus on an introduction to deep learning and explains core concepts. Meanwhile the intent behind the keyword query "Deep Learning" is find any papers that focus on deep learning.
            Your task is to semanticly interpret a question query, which is below delimited by triple quotes, and return three to six expanded queries so I can gather the score for each query and then aggregate over the scores to calculate a final score. Here are some examples:
            For the question query 'What is deep learning' the intent is to find papers that introduce and explain deep learning. At the end of each query please add a '$'.
            Therefore expanded querys could be:
            'Introduction to deep Learning$'
            'Overview of deep Learning$'
            'basic concepts of deep learning$'
            For the question query 'Why use boolean retrieval model' the inent is to find papers that gve insights to reasons to apply the boolean retrieval model. Therefore expanded querys could be:
            'Reasons to use boolean retrieval model$'
            'Introduction to boolean retrieval model$'

            query: '''{query}'''
            """
            new_query = ask_gpt(prompt=expand).lower().strip().replace("'", " ").replace('"', ' ')
            print("New query: ",new_query)
            questions.append({'qid':qid,'query': query,'subqueries': new_query})

Old query:  what makes natural language processing natural
New query:   fundamentals of natural language processing$ 
 understanding natural language processing$ 
 concepts behind natural language processing$ 
 principles of natural language processing$ 
 introduction to natural language processing$ 
Old query:  what is ahp
New query:   introduction to ahp$ 
 overview of ahp$ 
 basic concepts of ahp$ 
 understanding ahp$ 
 principles of ahp$ 
Old query:  what is information retrieval
New query:   introduction to information retrieval$ 
 overview of information retrieval$ 
 basic concepts of information retrieval$ 
 principles of information retrieval$ 
 understanding information retrieval$ 


In [31]:
print(questions)

[{'qid': '56', 'query': 'what makes natural language processing natural', 'subqueries': ' fundamentals of natural language processing$ \n understanding natural language processing$ \n concepts behind natural language processing$ \n principles of natural language processing$ \n introduction to natural language processing$ '}, {'qid': '59', 'query': 'what is ahp', 'subqueries': ' introduction to ahp$ \n overview of ahp$ \n basic concepts of ahp$ \n understanding ahp$ \n principles of ahp$ '}, {'qid': '60', 'query': 'what is information retrieval', 'subqueries': ' introduction to information retrieval$ \n overview of information retrieval$ \n basic concepts of information retrieval$ \n principles of information retrieval$ \n understanding information retrieval$ '}]


In [34]:
#temporary
questions = [{'qid': '56', 'query': 'what makes natural language processing natural', 'subqueries': ' fundamentals of natural language processing$ \n understanding natural language processing$ \n concepts behind natural language processing$ \n principles of natural language processing$ \n introduction to natural language processing$ '}, {'qid': '59', 'query': 'what is ahp', 'subqueries': ' introduction to ahp$ \n overview of ahp$ \n basic concepts of ahp$ \n understanding ahp$ \n principles of ahp$ '}, {'qid': '60', 'query': 'what is information retrieval', 'subqueries': ' introduction to information retrieval$ \n overview of information retrieval$ \n basic concepts of information retrieval$ \n principles of information retrieval$ \n understanding information retrieval$ '}]

In [35]:
for item in questions:
    item['subqueries'] = item['subqueries'].replace('\n', ' ').split('$')
    del item['subqueries'][-1] #last item in list would be empty string, remove it
print(questions)

[{'qid': '56', 'query': 'what makes natural language processing natural', 'subqueries': [' fundamentals of natural language processing', '   understanding natural language processing', '   concepts behind natural language processing', '   principles of natural language processing', '   introduction to natural language processing']}, {'qid': '59', 'query': 'what is ahp', 'subqueries': [' introduction to ahp', '   overview of ahp', '   basic concepts of ahp', '   understanding ahp', '   principles of ahp']}, {'qid': '60', 'query': 'what is information retrieval', 'subqueries': [' introduction to information retrieval', '   overview of information retrieval', '   basic concepts of information retrieval', '   principles of information retrieval', '   understanding information retrieval']}]


now do the ranking of each subquery


In [36]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [75]:
import pandas as pd
final_score= list()
for query in questions: #iterate list to get dict of query
    qscores = list()
    qid = query['qid']
    old_query = query['query']
    for i,subquery in enumerate(query['subqueries']): #iterate through subqueries key of dict
        results = bm25.search(subquery) #rank each subquery, returns pd dataframe
        for index, row in results.iterrows(): #iterate through dataframe and format it
            docid = row['docid']
            docno = row['docno']
            score = row['score']
            df_row = {'qid':qid, 'docid':docid, 'docno':docno, 'rank': index, 'score': score, 'query': subquery}
            qscores.append(df_row)
    final_score.append(qscores)
df = pd.DataFrame(final_score[0])
print(len(df))

5000


In [81]:
#construct dataframe for each query
dataframes = list()
for i in range(len(final_score)):
    dataframes.append(pd.DataFrame(final_score[i]))

5000
4997
5000


In [112]:
dataframes[0].head(3)

,qid,docid,docno,rank,score,query
0,56,69635,C82-2020,0,18.680956,fundamentals of natural language processing
1,56,48955,J15-1007,1,18.662827,fundamentals of natural language processing
2,56,36863,R09-1050,2,18.205316,fundamentals of natural language processing


In [120]:
query_dataframe = list()
seen_docnos = list()
for index, row in dataframes[0].iterrows():
    docno = row['docno']
    if docno not in seen_docnos:
        seen_docnos.append(docno)
        indices = df.index[df['docno'] == docno].tolist()
        sum_score = 0
        for i in indices:
            score = df.loc[i, 'score']
            sum_score += score
        query_dataframe.append({'qid':df.loc[i, 'qid'], 'docid': df.loc[i, 'docid'], 'docno': docno,'rank':0, 'score': sum_score, 'query': df.loc[i, 'query']})
    else:
        continue

In [121]:
df0 = pd.DataFrame(query_dataframe)
df0.head(3)

,qid,docid,docno,rank,score,query
0,56,69635,C82-2020,0,48.364692,introduction to natural language processing
1,56,48955,J15-1007,0,67.541749,introduction to natural language processing
2,56,36863,R09-1050,0,69.011352,introduction to natural language processing


In [103]:
#now merge each dataframe

#have a list of dicts where you store the qid, docno etc. as well as final score
#loop through dataframe and get docno
#if the docno not already in dict then look for all indices that have this docno
#sum the score of each indice and add the docno with the final score to the dict
#if the docno is already in the dict then skip
df = dataframes[0]
final_df = dict()
df.head(3)
indices = df.index[df['docno'] == 'C82-2020'].tolist()
print(df.loc[0, 'score'])
print(df.loc[3792, 'score'])
print(indices)
#df = df.to_dict()

18.680955810882573
9.894578773515217
[0, 2942, 3792, 4885]


In [43]:
#example of how to create dataframe
import pandas as pd
data = [
    {"qid": 1, "docid": 101, "docno": "D1", "rank": 1, "score": 0.95, "query": "example query 1"},
    {"qid": 1, "docid": 102, "docno": "D2", "rank": 2, "score": 0.85, "query": "example query 1"}
]
df = pd.DataFrame(data)
df.head(2)

,qid,docid,docno,rank,score,query
0,1,101,D1,1,0.95,example query 1
1,1,102,D2,2,0.85,example query 1
